# 1ª Tarea

Es necesario seleccionar un dataset para aplicar la técnica de Reglas de Asociación. El Dataset elegido fue: http://archive.ics.uci.edu/ml/datasets/Dota2+Games+Results

# 2ª Tarea

## a) Descripción de dataset:

El dataset viene de un juego llamado "Dota 2", es un MOBA (Multiplayer Online Battle Arena o [Multijugador de Arena de Batalla en Línea](https://es.wikipedia.org/wiki/Videojuego_multijugador_de_arena_de_batalla_en_l%C3%ADnea)), donde tenemos 2 equipos que pelean para destruir la base enemiga. El Juego tiene 113 personajes y cada jugador puede eligir un personaje distinto, cada equipo tiene 5 jugadores, entonces al todo 10 de los 113 son elegidos.<br>

En su versión original, el dataset ofrece las informaciones relativas a una partida, las columnas son:<br>
1: Victoria (1) o Derrota (-1), no hay empate, ya que el juego solo termina cuando alguien destruye la base enemiga.<br>
2: Id de la región (el servidor en que se juega, por ejemplo: China, Korea del Sur, etc)<br>
3: Modo de juego (Como los héroes son elegidos)<br>
4: Gameplay (como el juego va a ser, contra bots, competición, etc)<br>
5-117: id de los héroes

## b) ¿Cual es el objetivo de aplicar la técnica de Reglas de Asociación?

El juego contiene 113 personajes e intenta hacer un equilibrio de personajes y oponentes, entonces cada partida debe haber una chance de 50% de chance de victoria y derrota, una vez que teóricamente todos los personajes son equilibrados y el mismo para los equipos. Pero ¿es posible encontrar una regla de un personaje o grupo de personajes que tiene más posibilidades de victoria?<br>

La idea es analizar diversos sub-datasets basados en ese para ver si hay alguna regla de confianza significativamente alta en que un (o más) personajes lleven a victoria o derrota.<br>

Pero ¿cual seria una confianza significativamente alta? En un caso de MOBA, cualquier personaje con más de 50% de victoria ya seria un valor significativo, pero el juego no es solo los personajes, aún hay la habilidad del jugador, entonces para definir una buena confianza es interesante mirar un sitio que hace tasas de victoria para cada personaje: [un para Dota](https://www.dotabuff.com/heroes/winning) y [Otro para un otro MOBA conocido, LoL](https://champion.gg/statistics/#?sortBy=general.winPercent&order=descend))<br>

Así vemos que la tasa de victoria o derrota común es entre 45%-55%, entonces un valor arriba o abajo deses es un valor considerable.

## c) Pre-Procesamiento

A principio los datos vienen en un formato más difícil de utilizar, entonces vamos a principio cambiar os datos de como fueran descritos en "a)" para un formato como:<br>

Victoria, Personaje A, Personaje B, Personaje C, Personaje D, Personaje E<br>
Derrota, Personaje F, Personaje G, Personaje H, Personaje I, Personaje J<br>

Cada línea genera dos líneas con 10 nombres de personajes distintos y una victoria y una derrota<br>
Pero aún hay diversos modos de juego, entonces podemos separarlos también en archivos por modo de juego e elección. Eso es importante por algunas razones:<br>


1. Es muy dificil que una persona que tiene total libertad de elección de personaje va eligir un personaje con el cual no sepa jugar, entonces hay menor probabilidad de no saber jugar hacer parte da decisión de Victoria o Derrota<br>
2. Juegos competitivos (Ranked) están dirigidos para las personas que tienen habilidad en un personaje específico y son mejor equilibrados (hay categorías diferentes para personas de habilidades diferentes)<br>
3. Selecciones aleatorias pueden favorecer una mejor tasa de victoria para personajes fáciles de aprender o peor para personajes difíciles.<br>

Por eso es mejor hacer la separación de datasets basados en las características 3 y 4<br>
Así sigue el código que hace la separación de dataset por modos de juego y gameplay:

In [1]:
import csv

def split_database(path):
    # Ese es el nombre base de los archivos que vamos a poner los dados 
    base_file_name = './dota2Dataset/dota2-'
    file_names = []
    files = {}
    
    # Vamos a hacer un diccionario con listas para poner los juegos
    for i in range(0, 23):
        for j in range(-1, 8):
            file_names.append('{}_{}'.format(i, j))
            files[file_names[-1]] = []
    
    # Ahora debemos abrir el archivo csv con todos los datos
    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar="'")
        
        for row in reader:
            gm = row[2]
            gt = row[3]
            
            if(gt.isdigit() and gm.isdigit()):
                # Y poner cada línea en su respectivo archivo en el diccionario
                files['{}_{}'.format(gm, gt)].append(row)
    
    # Ahora hacer un archivo con todos 
    for f_name in file_names:
        lines_len = len(files[f_name])
        header = 'Team Victory, Cluster ID, Game Mode, Game Type, Champion 1, Champion 2, Champion 3, Champion 4, Champion 5, Champion 6, Champion 7, Champion 8, Champion 9, Champion 10, Champion 11, Champion 12, Champion 13, Champion 14, Champion 15, Champion 16, Champion 17, Champion 18, Champion 19, Champion 20, Champion 21, Champion 22, Champion 23, Champion 24, Champion 25, Champion 26, Champion 27, Champion 28, Champion 29, Champion 30, Champion 31, Champion 32, Champion 33, Champion 34, Champion 35, Champion 36, Champion 37, Champion 38, Champion 39, Champion 40, Champion 41, Champion 42, Champion 43, Champion 44, Champion 45, Champion 46, Champion 47, Champion 48, Champion 49, Champion 50, Champion 51, Champion 52, Champion 53, Champion 54, Champion 55, Champion 56, Champion 57, Champion 58, Champion 59, Champion 60, Champion 61, Champion 62, Champion 63, Champion 64, Champion 65, Champion 66, Champion 67, Champion 68, Champion 69, Champion 70, Champion 71, Champion 72, Champion 73, Champion 74, Champion 75, Champion 76, Champion 77, Champion 78, Champion 79, Champion 80, Champion 81, Champion 82, Champion 83, Champion 84, Champion 85, Champion 86, Champion 87, Champion 88, Champion 89, Champion 90, Champion 91, Champion 92, Champion 93, Champion 94, Champion 95, Champion 96, Champion 97, Champion 98, Champion 99, Champion 100, Champion 101, Champion 102, Champion 103, Champion 104, Champion 105, Champion 106, Champion 107, Champion 108, Champion 109, Champion 110, Champion 111, Champion 112, champion 113'
        
        # Con eso solo vamos a hacer archivos para los que tienen datos
        if(lines_len > 0):
            with open(base_file_name + f_name + '.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile, delimiter=',', quotechar="'")
                writer.writerow([header])
                
                for row in files[f_name]:
                    writer.writerow(row)

Ahora que los archivos están divididos, necesitamos cambiar esas informaciones para el modelo dicho anteriormente, para eso vamos a usar la función:

In [2]:
import json

def get_team_comp_data(data_frame):
    # Primero vamos a hacer una lista con el nombre de los héroes para cambiar los identificadores por los nombres
    heroes = ['']*113
    heroes_file = open('./dota2Dataset/heroes.json', 'r')
    json_cont = json.load(heroes_file)
    
    for element in json_cont['heroes']:
        heroes[element['id'] - 1] = element['localized_name'].replace(' ', '_').replace("'", '')
    
    new_data = []
    
    rows_num = data_frame.shape[0]
    perc = 0
    last_perc = -1
    
    # Ahora por cada línea vamos a poner el resultado del equipo y los personajes usados
    for index, row in data_frame.iterrows():
        perc = index*100.0/rows_num
        
        if(int(perc)%30 == 0 and last_perc != int(perc)):
            print("{}%".format(perc))
            last_perc = int(perc)
            
        new_row_t1 = []
        new_row_t2 = []
        victory_t1 = "Victory" if row['Team Victory'] == 1 else "Defeat"
        victory_t2 = "Victory" if row['Team Victory'] == -1 else "Defeat"
        
        new_row_t1.append(victory_t1)
        new_row_t2.append(victory_t2)
        
        for i in range(113):
            
            if(row.iloc[4 + i] == 1):
                new_row_t1.append(heroes[i])
            elif(row.iloc[4 + i] == -1):
                new_row_t2.append(heroes[i])
        
        new_data.append(new_row_t1)
        new_data.append(new_row_t2)
        
    return new_data

Asi es posible ejecutar eses codigos y generar las seguintes salidas:

In [3]:
import os

print('Los archivos en el directorio dota2Dataset son: \n')
for file in os.listdir('dota2Dataset'):
    print(file)
    
path = './dota2Dataset/dota2Full.csv'
split_database(path)

print('\nAhora los archivos en el directorio dota2Dataset son: \n')
for file in os.listdir('dota2Dataset'):
    print(file)

Los archivos en el directorio dota2Dataset son: 

dota2Full.csv
dota2FullChanged.csv
dota2FullChanged_ant.csv
dota2Test.csv
dota2Train.csv
heroes.json

Ahora los archivos en el directorio dota2Dataset son: 

dota2-1_2.csv
dota2-2_2.csv
dota2-2_3.csv
dota2-3_2.csv
dota2-4_2.csv
dota2-5_2.csv
dota2-6_1.csv
dota2-6_2.csv
dota2-6_3.csv
dota2-7_2.csv
dota2-8_2.csv
dota2-8_3.csv
dota2-9_1.csv
dota2-9_2.csv
dota2Full.csv
dota2FullChanged.csv
dota2FullChanged_ant.csv
dota2Test.csv
dota2Train.csv
heroes.json


Abajo la lista de nuevos archivos, que tienen como nombre dota2-MODO_DE_JUEGO_GAMEPLAY, explanados entre parenteses:<br>

dota2-1_2.csv (All Pick, Tournament. Torneo de libre elección, 1365 juegos)<br> 
dota2-2_2.csv (Captains Mode, Tournament. Torneo de elección del capitán, 45713 juegos)<br>
dota2-2_3.csv (Captains Mode, Tutorial. Tutorial de elección del capitán, 34139 juegos)<br>
dota2-3_2.csv (Random Draft, Tournament. Torneo de elección aleatoria entre 50 héroes, 434 juegos)<br>
dota2-4_2.csv (Single Draft, Tournament. Torneo de elección aleatoria entre 3 héroes, 226 juegos)<br>
dota2-5_2.csv (All Random, Tournament. Torneo de elección aleatoria, 13 juegos)<br>
dota2-6_1.csv (Deathmatch, Practice.  Práctica de partida de muerte, 19 juegos)<br>
dota2-6_2.csv (Deathmatch, Tournament. Torneo de partida de muerte, 315 juegos)<br>
dota2-6_3.csv (Deathmatch, Tutorial.  Tutorial de partida de muerte, 220 juegos)<br>
dota2-7_2.csv (The Diretide, Tournament. Torneo Diretide, 97 juegos)<br>
dota2-8_2.csv (Reverse Captains Mode, Tournament. Torneo de elección del capitán reverso, 3910 juegos)<br>
dota2-8_3.csv (Reverse Captains Mode, Tutorial.  Tutorial de elección del capitán reverso, 5293 juegos)<br>
dota2-9_1.csv (Greeviling, Practice. Práctica de Greeviling , 1 juego)<br>
dota2-9_2.csv (Greeviling, Tournament. Torneo de Greeviling, 11199 juegos)<br>

Pero algunos de los modos de juego anteriores no tienen muchos datos. Es seguro afirmar que con 113 héroes, es necesario más que 1000 partidas para tenemos una buena analice. Entonces nos quedariamos con:<br>

dota2-2_2.csv (Captains Mode, Tournament. Torneo de elección del capitán, 45713 juegos)<br>
dota2-2_3.csv (Captains Mode, Tutorial. Tutorial de elección del capitán, 34139 juegos)<br>
dota2-9_2.csv (Greeviling, Tournament. Torneo de Greeviling, 11199 juegos)<br>
dota2-8_3.csv (Reverse Captains Mode, Tutorial.  Tutorial de elección del capitán reverso, 5293 juegos)<br>
dota2-8_2.csv (Reverse Captains Mode, Tournament. Torneo de elección del capitán reverso, 3910 juegos)<br>
dota2-1_2.csv (All Pick, Tournament. Torneo de libre elección, 1365 juegos)<br> 

Así tenemos esas bases de datos. Las cosas a ser consideradas son:
1. En la elección del capitán una persona del equipo elige cual son los héroes que los jugadores de su equipo poden utilizar
2. En la elección del capitán reversa, el capitán va eligir cual son los héroes que sus enemigos tendrán que utilizar
3. En Greeviling y All Pick, todos los jugadores pueden eligir los personajes que quieren
4. Un Torneo tendrá los mejores jugadores, entonces hay un equilibrio de nivel en las partidas
5. Un equipo de torneo posiblemente tendrá una buena comunicación, entonces el capitán va eligir los héroes que su equipo sabe jugar

Ahora vamos a ver la transformación de las bases para el otro modelo

In [4]:
import pandas as pd

data_df_full = pd.read_csv("./dota2Dataset/dota2Full.csv", index_col=False)
data_df__2_2 = pd.read_csv("./dota2Dataset/dota2-2_2.csv", index_col=False)
data_df__2_3 = pd.read_csv("./dota2Dataset/dota2-2_3.csv", index_col=False)
data_df__9_2 = pd.read_csv("./dota2Dataset/dota2-9_2.csv", index_col=False)
data_df__8_3 = pd.read_csv("./dota2Dataset/dota2-8_3.csv", index_col=False)
data_df__8_2 = pd.read_csv("./dota2Dataset/dota2-8_2.csv", index_col=False)
data_df__1_2 = pd.read_csv("./dota2Dataset/dota2-1_2.csv", index_col=False)

datas_df = [data_df_full, data_df__2_2, data_df__2_3, data_df__9_2, data_df__8_3, data_df__8_2, data_df__1_2]
datas_conv = []

for i in range(len(datas_df)):
    print('Converting file {}/{}'.format(i+1, len(datas_df)))
    datas_conv.append(get_team_comp_data(datas_df[i]))

Converting file 1/7
0.0%
30.00077712154181%
60.00058284115636%
90.0003885607709%
Converting file 2/7
0.0%
30.000218756152517%
60.000437512305034%
90.00065626845755%
Converting file 3/7
0.0%
30.00087876036205%
60.0017575207241%
90.00263628108615%
Converting file 4/7
0.0%
30.00267881060809%
60.00535762121618%
90.00803643182427%
Converting file 5/7
0.0%
30.00188928773852%
60.00377857547704%
90.00566786321556%
Converting file 6/7
0.0%
30.0%
60.0%
90.0%
Converting file 7/7
0.0%
30.036630036630036%
60.0%
90.03663003663004%


In [5]:
for i in range(len(datas_conv)):
    print('File {}/{}: \n{}'.format(i+1, len(datas_conv), datas_conv[i][0]))

File 1/7: 
['Defeat', 'Morphling', 'Pudge', 'Lina', 'Slardar', 'Riki']
File 2/7: 
['Defeat', 'Morphling', 'Pudge', 'Lina', 'Slardar', 'Riki']
File 3/7: 
['Defeat', 'Sand_King', 'Lich', 'Necrophos', 'Phantom_Assassin', 'Alchemist']
File 4/7: 
['Victory', 'Shadow_Shaman', 'Skeleton_King', 'Death_Prophet', 'Alchemist', 'Bristleback']
File 5/7: 
['Victory', 'Pudge', 'Zeus', 'Necrophos', 'Slark', 'Centaur_Warrunner']
File 6/7: 
['Defeat', 'Shadow_Fiend', 'Pudge', 'Zeus', 'Faceless_Void', 'Rubick']
File 7/7: 
['Victory', 'Sven', 'Phantom_Assassin', 'Leshrac', 'Chaos_Knight', 'Treant_Protector']


## d) Obtener reglas de asociación

Tenemos muchos héroes, entonces el soporte tiene que ser muy bajo, buenos resultados aparecerán con el soporte en 1% (0.01). Y la confianza, pelo que vimos en el tópico anterior, una tasa de victoria arriba de 55% es considerable, entonces utilizaremos eses parámetros.

In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

def frequent_items(dataset, support=0.1):
    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df, min_support=support, use_colnames=True)
    
    return frequent_itemsets

def assoc_rul(frequent_itemsets, thresh):
    return association_rules(frequent_itemsets, metric="confidence", min_threshold=thresh)

In [7]:
confidences = []

for i in range(len(datas_conv)):
    print('File {}/{}: \nDoing frequent Itens'.format(i+1, len(datas_conv)))
    freq = frequent_items(datas_conv[i], 0.01)
    print('Filtering by confidence')
    conf = assoc_rul(freq, 0.56)
    
    confidences.append(conf)

File 1/7: 
Doing frequent Itens
Filtering by confidence
File 2/7: 
Doing frequent Itens
Filtering by confidence
File 3/7: 
Doing frequent Itens
Filtering by confidence
File 4/7: 
Doing frequent Itens
Filtering by confidence
File 5/7: 
Doing frequent Itens
Filtering by confidence
File 6/7: 
Doing frequent Itens
Filtering by confidence
File 7/7: 
Doing frequent Itens
Filtering by confidence


In [8]:
limit = 3
file_names = ['All Games', 'Captains Mode, Tournament', 'Captains Mode, Tutorial', 'Greeviling, Tournament', 'Reverse Captains Mode, Tutorial', 'Reverse Captains Mode, Tournament', 'All Pick, Tournament']

for i in range(len(confidences)):
    head = confidences[i].sort_values('confidence', ascending=False).head(limit)
    h = head[['antecedents','consequents', 'confidence']]
    print('{}:'.format(file_names[i]))
    print('{}\n'.format(h.to_string(index=False)))

All Games:
           antecedents consequents  confidence
          (Omniknight)   (Victory)    0.607665
 (Mirana, Drow_Ranger)   (Victory)    0.602620
       (Mirana, Pudge)   (Victory)    0.590532

Captains Mode, Tournament:
       antecedents consequents  confidence
      (Omniknight)   (Victory)    0.607781
     (Elder_Titan)   (Victory)    0.601012
 (Natures_Prophet)    (Defeat)    0.588611

Captains Mode, Tutorial:
           antecedents consequents  confidence
          (Omniknight)   (Victory)    0.612069
 (Mirana, Drow_Ranger)   (Victory)    0.609184
     (Natures_Prophet)    (Defeat)    0.605997

Greeviling, Tournament:
  antecedents consequents  confidence
 (Lone_Druid)    (Defeat)    0.636364
      (Meepo)    (Defeat)    0.594241
 (Omniknight)   (Victory)    0.585961

Reverse Captains Mode, Tutorial:
           antecedents consequents  confidence
         (Bristleback)    (Defeat)    0.639423
 (Mirana, Drow_Ranger)   (Victory)    0.635922
          (Omniknight)   (Victory) 

## e) Determinar posprocesamiento

Parte del posprocesamiento ya fue aplicado al decidir una confianza de más de 55%. El sitio en b) hace una clasificación basada en general, entonces con nuestro dataset es posible hacer una evaluación situacional. Vamos a enfocar en las dobles con mayor tasa de victoria primero, después en los personajes únicos con altas tasas de victoria.

In [9]:
for i in range(len(confidences)):
    
    sort = confidences[i].sort_values('confidence', ascending=False)
    pairs = []
    solos = []
    
    for j in range(len(sort['antecedents'])):
        if(len(sort['antecedents'].iloc[j]) > 1):
            pairs.append(sort.iloc[j])
        else:
            solos.append(sort.iloc[j])
    
    df_pairs = pd.DataFrame(pairs)[['antecedents', 'consequents', 'confidence']] if len(pairs) > 0 else None
    df_solos = pd.DataFrame(solos)[['antecedents', 'consequents', 'confidence']] if len(solos) > 0 else None
    print('{}::'.format(file_names[i]))
    print('Pairs:\n{}\n'.format(df_pairs))
    print('Solos:\n{}\n\n'.format(df_solos))

All Games::
Pairs:
              antecedents consequents  confidence
9   (Mirana, Drow_Ranger)   (Victory)    0.602620
11        (Mirana, Pudge)   (Victory)    0.590532
10   (Juggernaut, Mirana)   (Victory)    0.572220

Solos:
         antecedents consequents  confidence
8       (Omniknight)   (Victory)    0.607665
6      (Elder_Titan)   (Victory)    0.586554
2  (Natures_Prophet)    (Defeat)    0.586228
4             (Tusk)    (Defeat)    0.580413
3    (Queen_of_Pain)    (Defeat)    0.579854
0     (Ember_Spirit)    (Defeat)    0.578770
7        (Necrophos)   (Victory)    0.563792
5       (Windranger)    (Defeat)    0.562687
1             (Lina)    (Defeat)    0.561527


Captains Mode, Tournament::
Pairs:
                 antecedents consequents  confidence
11      (Juggernaut, Mirana)   (Victory)    0.581423
12           (Mirana, Pudge)   (Victory)    0.574478
13  (Riki, Phantom_Assassin)   (Victory)    0.567646

Solos:
            antecedents consequents  confidence
9          (Omnikn

## f) Analizar las reglas de asociación obtenidas

Con eso es posible ver que en dobles, la mejor opción es Mirana, en cualquier modo. En otros modos hay opciones más específicas para jugar solo. Por ejemplo: En partidas como Captain's mode en Torneo, los mejores son Omniknight y Elder Titan, ya Nature's Prophet, Ember Spirit, Queen of Pain y Tusk no son los mejores para se eligir nese modo.</br>

En otro modo, como Reverse Captain's Mode, donde una persona elige los personajes del adversario, puede ser mejor que el elija Bristleback para su enemigo y no elija Omniknight.

## g) Mostrar un ejemplo de cómo los resultados obtenidos pueden ser usados

Generalmente ese tipo de juegos tienen muchos patches (actualizaciones) para hacer correcciones y otras cosas nuevas, entonces conseguindo las informaciones y usando eses códigos puede hacer recomendaciones fáciles para nuevos jugadores o la propria empresa del juego puede ver si necesita cambiar algo en algun personaje, para hacerlo más equilibrado